In [97]:
import requests
import json
import re
import os
import pandas as pd

df = pd.read_csv('../temp/intermediate_mapping_file.csv') ###### Make input later

cases_endpt = "https://api.gdc.cancer.gov/cases"

# This set of filters is nested under an 'and' operator.
# Filtering for TCGA Masked Somatic Mutation - results in open access maf files.
filters = {
    "op": "and",
    "content": [
        {
            "op": "in",
            "content": {
                "field": "aliquot_ids",
                "value": "d7593b2a-0d86-44aa-a404-7fd1b10f65d4" # random from df 
            }
        }
    ]
}
# Here a GET is used, so the filter parameters should be passed as a JSON string.
fields = {
    "submitter_id",
    "case_id",
    "samples.sample_id",
    "samples.sample_type"
}
fields = ",".join(fields)

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "JSON",
    "size": "100",
    "pretty": "true"
}

response = requests.get(cases_endpt, params=params)
r = response.content.decode("utf-8")
print(r)

{
  "data": {
    "hits": [
      {
        "id": "edc62c30-cacc-457d-96ba-d05ed35432a3", 
        "case_id": "edc62c30-cacc-457d-96ba-d05ed35432a3", 
        "submitter_id": "TCGA-OR-A5K0", 
        "samples": [
          {
            "sample_id": "1e6b9fc4-3fed-4d32-89b8-aa15fb4a5840", 
            "sample_type": "Primary Tumor", 
            "tumor_code": null
          }, 
          {
            "sample_id": "37448d7e-eb2a-4f9e-9954-1f1597f45469", 
            "sample_type": "Primary Tumor"
          }, 
          {
            "sample_id": "ebab4f9f-0abf-4cf4-9a6d-bcc303e5ce43", 
            "sample_type": "Blood Derived Normal", 
            "tumor_code": null
          }
        ]
      }
    ], 
    "pagination": {
      "count": 1, 
      "total": 1, 
      "size": 100, 
      "from": 0, 
      "sort": "", 
      "page": 1, 
      "pages": 1
    }
  }, 
  "warnings": {}
}


In [72]:
print(df["Tumor_Sample_UUID"][1])

d7593b2a-0d86-44aa-a404-7fd1b10f65d4


KeyError: 'data'